In [28]:
import numpy as np
from tabulate import tabulate
import random
import heapq
import sys

# Symbols for the grid elements
wallSymbol = str('\U00002B1B')
floorSymbol = str('\U00002B1C')
botSymbol = str('\U0001F916')
buttonSymbol = str('\U0001F7E5')
fireSymbol = str('\U0001F525')

D = 40  # Ship dimensions
q = 1

# Global variable to choose which bot to run (1 for Bot 1, 2 for Bot 2)
bot_choice = 1  # Change this value to 1 for Bot 1 or 2 for Bot 2


# Function to generate the ship layout
def theShip(D, opencells):
    options = []
    ship = np.full((D, D), int(0), dtype=object)
    X, Y = random.randrange(D), random.randrange(D)  # Select a random point on the ship
    opencells.append(tuple([X, Y]))

    markOpen(ship, X, Y, D, options)

    while len(options) >= 1:
        rand = random.randrange(len(options))
        X, Y = options[rand]
        opencells.append(tuple([X, Y]))
        options = markOpen(ship, X, Y, D, options)

    # Detect and handle dead ends
    deadends = []
    for X in range(D):
        for Y in range(D):
            if isinstance(ship[X][Y], int):
                ship[X][Y] = wallSymbol
            if isinstance(ship[X][Y], str):
                isDeadend = 0
                if X - 1 >= 0:
                    if ship[X - 1][Y] == floorSymbol:
                        isDeadend += 1
                if X + 1 < D:
                    if ship[X + 1][Y] == floorSymbol:
                        isDeadend += 1
                if Y - 1 >= 0:
                    if ship[X][Y - 1] == floorSymbol:
                        isDeadend += 1
                if Y + 1 < D:
                    if ship[X][Y + 1] == floorSymbol:
                        isDeadend += 1
                if isDeadend == 1:
                    deadends.append(tuple([X, Y]))

    random.shuffle(deadends)

    for i in range(int(len(deadends) / 2)):
        X, Y = deadends[i]
        randselect = []
        randnum = 0
        if Y - 1 >= 0:
            if isinstance(ship[X][Y - 1], int):
                randselect.append(ship[X][Y - 1])
                randnum += 1
        if X - 1 >= 0:
            if isinstance(ship[X - 1][Y], int):
                randselect.append(ship[X - 1][Y])
                randnum += 1
        if X + 1 < D:
            if isinstance(ship[X + 1][Y], int):
                randselect.append(ship[X + 1][Y])
                randnum += 1
        if Y + 1 < D:
            if isinstance(ship[X][Y + 1], int):
                randselect.append(ship[X][Y + 1])
                randnum += 1

        if randnum > 0:
            randnum = random.randrange(randnum)
            deadends.remove(tuple([X, Y]))
            if Y - 1 >= 0 and isinstance(ship[X][Y - 1], int):
                if randnum == 0:
                    ship[X][Y - 1] = str(floorSymbol)
                    opencells.append(tuple([X, Y - 1]))
                randnum -= 1
            if X - 1 >= 0 and isinstance(ship[X - 1][Y], int):
                if randnum == 0:
                    ship[X - 1][Y] = str(floorSymbol)
                    opencells.append(tuple([X - 1, Y]))
                randnum -= 1
            if X + 1 < D and isinstance(ship[X + 1][Y], int):
                if randnum == 0:
                    ship[X + 1][Y] = str(floorSymbol)
                    opencells.append(tuple([X + 1, Y]))
                randnum -= 1
            if Y + 1 < D and isinstance(ship[X][Y + 1], int):
                if randnum == 0:
                    ship[X][Y + 1] = str(floorSymbol)
                    opencells.append(tuple([X, Y + 1]))

    print(tabulate(ship))
    return opencells, ship


# Function to open the cell and add its neighbors to the options list
def markOpen(ship, X, Y, D, options):
    ship[X][Y] = str(floorSymbol)

    if (X, Y) in options:
        options.remove(tuple([X, Y]))

    if Y - 1 >= 0 and isinstance(ship[X][Y - 1], int):
        ship[X][Y - 1] += 1
        if ship[X][Y - 1] == 1:
            options.append(tuple([X, Y - 1]))
        elif ship[X][Y - 1] > 1 and (X, Y - 1) in options:
            options.remove(tuple([X, Y - 1]))
    if X - 1 >= 0 and isinstance(ship[X - 1][Y], int):
        ship[X - 1][Y] += 1
        if ship[X - 1][Y] == 1:
            options.append(tuple([X - 1, Y]))
        elif ship[X - 1][Y] > 1 and (X - 1, Y) in options:
            options.remove(tuple([X - 1, Y]))
    if X + 1 < D and isinstance(ship[X + 1][Y], int):
        ship[X + 1][Y] += 1
        if ship[X + 1][Y] == 1:
            options.append(tuple([X + 1, Y]))
        elif ship[X + 1][Y] > 1 and (X + 1, Y) in options:
            options.remove(tuple([X + 1, Y]))
    if Y + 1 < D and isinstance(ship[X][Y + 1], int):
        ship[X][Y + 1] += 1
        if ship[X][Y + 1] == 1:
            options.append(tuple([X, Y + 1]))
        elif ship[X][Y + 1] > 1 and (X, Y + 1) in options:
            options.remove(tuple([X, Y + 1]))

    return options


# Place the bot, button, and fire
def placeObjects(bot, button, fire0, ship, opencells, fireNeighbors):
    ship[bot[0]][bot[1]] = botSymbol
    ship[button[0]][button[1]] = buttonSymbol
    fireNeighbors.append(tuple([fire0[0], fire0[1]]))
    fireNeighbors, opencells = startFire(fire0, ship, opencells, fireNeighbors)
    print(tabulate(ship))
    return fireNeighbors, opencells


# Fire spread logic
def spreadFire(fireNeighbors, opencells, ship):
    toFire = []

    for i in fireNeighbors:
        K = 0
        if i[0] - 1 >= 0 and ship[i[0] - 1][i[1]] == fireSymbol:
            K += 1
        if i[0] + 1 < D and ship[i[0] + 1][i[1]] == fireSymbol:
            K += 1
        if i[1] - 1 >= 0 and ship[i[0]][i[1] - 1] == fireSymbol:
            K += 1
        if i[1] + 1 < D and ship[i[0]][i[1] + 1] == fireSymbol:
            K += 1
        p = (1 - ((1 - q) ** K))
        if random.uniform(0, 1) <= p:
            toFire.append(tuple(i))

    for i in toFire:
        startFire(i, ship, opencells, fireNeighbors)

    return opencells, fireNeighbors


# Fire ignition logic
def startFire(Fire, ship, opencells, fireNeighbors):
    ship[Fire[0]][Fire[1]] = fireSymbol
    fireNeighbors.remove(tuple([Fire[0], Fire[1]]))
    opencells.remove(tuple([Fire[0], Fire[1]]))
    if Fire[1] - 1 >= 0 and (ship[Fire[0]][Fire[1] - 1] in [floorSymbol, botSymbol, buttonSymbol]):
        fireNeighbors.append(tuple([Fire[0], Fire[1] - 1]))
    if Fire[1] + 1 < D and (ship[Fire[0]][Fire[1] + 1] in [floorSymbol, botSymbol, buttonSymbol]):
        fireNeighbors.append(tuple([Fire[0], Fire[1] + 1]))
    if Fire[0] + 1 < D and (ship[Fire[0] + 1][Fire[1]] in [floorSymbol, botSymbol, buttonSymbol]):
        fireNeighbors.append(tuple([Fire[0] + 1, Fire[1]]))
    if Fire[0] - 1 >= 0 and (ship[Fire[0] - 1][Fire[1]] in [floorSymbol, botSymbol, buttonSymbol]):
        fireNeighbors.append(tuple([Fire[0] - 1, Fire[1]]))
    return fireNeighbors, opencells


# Heuristic function for A*
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


# Get neighbors and avoid fire and walls
def get_neighbors(ship, node, fire, fireNeighbors):
    neighbors = []
    x, y = node
    potential_moves = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]  # N, S, W, E
    if bot_choice != 3:
        for nx, ny in potential_moves:
            if 0 <= nx < D and 0 <= ny < D and ship[nx][ny] != wallSymbol and ship[nx][ny] != fireSymbol:
                neighbors.append((nx, ny))
    if bot_choice == 3:
        for nx, ny in potential_moves:
            if 0 <= nx < D and 0 <= ny < D and ship[nx][ny] != wallSymbol and ship[nx][ny] != fireSymbol and ([nx, ny]) != fireNeighbors:
                neighbors.append((nx, ny))
    return neighbors


# A* algorithm for the bot
def a_star(ship, start, goal, fire, fireNeighbors):
    close_set = set()
    came_from = {}
    gscore = {start: 0}
    fscore = {start: heuristic(start, goal)}

    open_heap = []
    heapq.heappush(open_heap, (fscore[start], start))

    while open_heap:
        current = heapq.heappop(open_heap)[1]

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path = path[::-1]
            return path

        close_set.add(current)

        for neighbor in get_neighbors(ship, current, fire, fireNeighbors):
            tentative_gscore = gscore[current] + 1

            if neighbor in close_set and tentative_gscore >= gscore.get(neighbor, 0):
                continue

            if tentative_gscore < gscore.get(neighbor, float('inf')):
                came_from[neighbor] = current
                gscore[neighbor] = tentative_gscore
                fscore[neighbor] = tentative_gscore + heuristic(neighbor, goal)
                heapq.heappush(open_heap, (fscore[neighbor], neighbor))

    return False  # No path found


# Move the bot along the path
def bot_move(ship, bot, path, button):
    if ship[bot[0]][bot[1]] == fireSymbol:
        print("Can't move")
        sys.exit("FAILURE")
    if path:
        next_move = path.pop(0)
        ship[bot[0]][bot[1]] = floorSymbol  # Clear bot's previous position
        bot[0], bot[1] = next_move          # Move to the next position
        ship[bot[0]][bot[1]] = botSymbol    # Update the bot's new position
    if tuple(bot) == button:
        print("The bot reached the button!")
        sys.exit("SUCCESS")
    return bot, path


# Run either Bot 1 or Bot 2
def run_bot(ship, bot, button, fire0, opencells, fireNeighbors):
    # Bot (A*): Calculate the shortest path
    path = a_star(ship, tuple(bot), button, fire0, fireNeighbors)
    if not path:
        print("No path found for the bot!")
        sys.exit("FAILURE")

    print(f"Bot - Shortest Path: {path}")
    print(f"Length of the Shortest Path: {len(path)}")

    t = 75
    ti = 0
    while ti < t and path:
        ti += 1

        if tuple(bot) == button:
            print("The bot reached the button!")
            sys.exit("FAILURE")
        
        # Move the bot
        bot, path = bot_move(ship, bot, path, button)
        print("Bot moved:")
        print(tabulate(ship))

        # Spread fire at each time step
        opencells, fireNeighbors = spreadFire(fireNeighbors, opencells, ship)
        print(tabulate(ship))

        if bot_choice == 2 or bot_choice == 3:
            path = a_star(ship, tuple(bot), button, fire0, fireNeighbors)
            if not path:
                print("No path found for the bot!")
                sys.exit("FAILURE")
            
            print(f"Bot - Shortest Path: {path}")
            print(f"Length of the Shortest Path: {len(path)}")

        # Check if the bot reaches the button
       


# Main function: run based on global bot_choice variable
def main():
    opencells = []
    opencells, ship = theShip(D, opencells)
    random.shuffle(opencells)

    fireNeighbors = []
    bot = list(opencells[0])  # Bot's starting position
    button = opencells[1]     # Button's position
    fire0 = opencells[2]      # Initial fire's position

    placeObjects(bot, button, fire0, ship, opencells, fireNeighbors)

    run_bot(ship, bot, button, fire0, opencells, fireNeighbors)

if __name__ == "__main__":
    main()


--  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --
⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜
⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬛  ⬜
⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜
⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬛  ⬜  ⬜  ⬜
⬜  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛
⬛  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜
⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜
⬜  ⬛  ⬜ 

SystemExit: SUCCESS